In [2]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
import osmnx as ox
import networkx as nx
from tzwhere import tzwhere

In [3]:
data = pd.read_csv('../datasets/uber.csv',low_memory=False,index_col=0)
data = data[~((data['pickup_longitude']>-70.1614) | (data['pickup_longitude']<-76.2314) | (data['pickup_latitude']>42.2713) | (data['pickup_latitude']<39.6756))]
data = data[~((data['dropoff_longitude']>-70.1614) | (data['dropoff_longitude']<-76.2314) | (data['dropoff_latitude']>42.2713) | (data['dropoff_latitude']<39.6756))]
data = data.drop(columns=["key"])
data.pickup_datetime = pd.to_datetime(data.pickup_datetime)

In [4]:
date = [[time.year,time.quarter, time.month,time.day,time.dayofweek,time.hour] for time in data.pickup_datetime]

In [5]:
data [["year","quarter","month","day","dayofweek","hour"]] = date
data["isweekend"] = (data.dayofweek > 4)*1

In [6]:
data["pickuplocation"] = list(zip(data.pickup_latitude,data.pickup_longitude))
data["dropofflocation"] = list(zip(data.dropoff_latitude,data.dropoff_longitude))
data = data.drop(columns = ["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"])

In [7]:
pickup = np.vstack(data.pickuplocation)
dropoff = np.vstack(data.dropofflocation)
dis = np.sqrt((pickup[:,0] - dropoff[:,0])**2 + (pickup[:,1] - dropoff[:,1])**2)

In [8]:
data

,fare_amount,pickup_datetime,passenger_count,year,quarter,month,day,dayofweek,hour,isweekend,pickuplocation,dropofflocation
24238194,7.5,2015-05-07 19:52:06+00:00,1,2015,2,5,7,3,19,0,"(40.73835372924805, -73.99981689453125)","(40.72321701049805, -73.99951171875)"
27835199,7.7,2009-07-17 20:04:56+00:00,1,2009,3,7,17,4,20,0,"(40.728225, -73.994355)","(40.750325, -73.99471)"
44984355,12.9,2009-08-24 21:45:00+00:00,1,2009,3,8,24,0,21,0,"(40.74077, -74.005043)","(40.772647, -73.962565)"
25894730,5.3,2009-06-26 08:22:21+00:00,3,2009,2,6,26,4,8,0,"(40.790844, -73.976124)","(40.803349, -73.965316)"
17610152,16.0,2014-08-28 17:47:00+00:00,5,2014,3,8,28,3,17,0,"(40.744085, -73.925023)","(40.761247, -73.97308199999999)"
...,...,...,...,...,...,...,...,...,...,...,...,...
42598914,3.0,2012-10-28 10:49:00+00:00,1,2012,4,10,28,6,10,1,"(40.739367, -73.987042)","(40.740297, -73.986525)"
16382965,7.5,2014-03-14 01:09:00+00:00,1,2014,1,3,14,4,1,0,"(40.736837, -73.98472199999999)","(40.73962, -74.006672)"
27804658,30.9,2009-06-29 00:42:00+00:00,2,2009,2,6,29,0,0,0,"(40.756487, -73.986017)","(40.692588, -73.85895699999999)"
20259894,14.5,2015-05-20 14:56:25+00:00,1,2015,2,5,20,2,14,0,"(40.7254524230957, -73.99712371826173)","(40.69541549682617, -73.98321533203125)"


In [9]:
import osmnx as ox
import networkx as nx
from datetime import timedelta


In [14]:
pickup = np.vstack(data.pickuplocation)
dropoff = np.vstack(data.pickuplocation)
lats = np.hstack([pickup[:,0],dropoff[:,0]])
lons = np.hstack([pickup[:,1],dropoff[:,1]])

north,south,east,west = max(lats),min(lats),max(lons),min(lons)
ox.graph_from_bbox(north,south,east,west, network_type='all_private')

In [24]:
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

geometry1 = [Point(point) for point in data.pickuplocation]
geometry2 = [Point(point) for point in data.dropofflocation]
geometry = geometry1 

gdf = GeoDataFrame(data, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: DLL load failed while importing ogrext: The specified module could not be found.

In [23]:
df = gpd.read_file(gpd.datasets.get_path('nybb'))

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: DLL load failed while importing ogrext: The specified module could not be found.

In [26]:
data = pd.read_csv('../datasets/uber.csv',low_memory=False,index_col=0)
df = pd.DataFrame()
a = np.hstack((data[['pickup_longitude','pickup_latitude']],data[['dropoff_longitude','dropoff_latitude']]))
df[['lon','lat']] = np.array([[aa[0],aa[1]] for aa in a])

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = GeoDataFrame(df, geometry=geometry)   
#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

ImportError: the 'read_file' function requires the 'fiona' package, but it is not installed or does not import correctly.
Importing fiona resulted in: DLL load failed while importing ogrext: The specified module could not be found.